In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0
Q.<alpha> = NumberField(xx^2-5)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [ ]:
# Define cubic SE10, find and classify the lines and compute the 45 tritangent planes

In [4]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE10 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)}).subs({e : (2-alpha)*c})

In [ ]:
# Find simmetries

In [5]:
%time adm_SE10 = SE10.find_admissible_projectivities()
len(adm_SE10)

CPU times: user 593 ms, sys: 87.8 ms, total: 681 ms
Wall time: 2.65 s


120

In [6]:
%time simm_SE10 = SE10.find_simmetries(adm_SE10)
len(simm_SE10)

CPU times: user 430 ms, sys: 56.4 ms, total: 486 ms
Wall time: 636 ms


120

In [ ]:
# Study how the simmetries permute the Eckardt points

In [7]:
Eck_points_perms = []
for proj in simm_SE10:
    Eck_points_perms.append(Permutation(SE10.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

(120, 'S5')

In [ ]:
# We want to find what elements are permuted by the group S5
# These are the so called Sylvester planes

In [13]:
indices = []
for gen in Eck_points_perms_group.gens_small():
    indices.append(Eck_points_perms.index(gen))
gen1 = simm_SE10[indices[0]]
gen2 = simm_SE10[indices[1]]

In [14]:
Sylvester_planes_labels = [['E1', 'G3', 'F13'], ['E3', 'G4', 'F34'], ['E2', 'G1', 'F12'], ['E4', 'G2', 'F24'], ['F14', 'F23', 'F56']]

In [15]:
bool([label in SE10.eckardt_points_labels for label in Sylvester_planes_labels])

True

In [28]:
#Readjust the coefficients of the planes to match those of the cubic
Sylvester_planes = [pl.plane/min(pl.plane.coefficients()) for pl in SE10.tritangent_planes if pl.labels in Sylvester_planes_labels]
for i in range(len(Sylvester_planes)):
    if i in [0,1]:
        Sylvester_planes[i]*=-1
    if i in [2, 4]:
        Sylvester_planes[i]*=2

In [29]:
# Check that the sum of the planes cubed is actually SE10
mon = ((x+y+z+t)^3).monomials()
nc = sum([pl^3 for pl in Sylvester_planes])
[el for el in matrix([[nc.coefficient(mn) for mn in mon], [SE10.eqn.coefficient(mn) for mn in mon]]).minors(2) if el!=0]

[]

In [30]:
sost1 = change_coord(gen1)
sost2 = change_coord(gen2)

In [31]:
[True in [pl.subs(sost1).divides(pl1) for pl in Sylvester_planes] for pl1 in Sylvester_planes]

[True, True, True, True, True]

In [32]:
[True in [pl.subs(sost2).divides(pl1) for pl in Sylvester_planes] for pl1 in Sylvester_planes]

[True, True, True, True, True]

In [ ]:
# Study how the simmetries permute the 27 lines

In [23]:
%%time
lines_perms = []
for simm in simm_SE10:
    lines_perms.append(Permutation(SE10.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

CPU times: user 16.7 s, sys: 112 ms, total: 16.8 s
Wall time: 16.8 s


(120, 'S5')